In [5]:
import re

from collections import Counter

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Input, GRU, LSTM


from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

# Работаем с датаестом

In [3]:
data=pd.read_csv("https://raw.githubusercontent.com/Terkingil/Project_NLP/main/Models%20weights%20n%20data/S_C_model/NE%20or%20abra.csv")
display(data)

,Name,Label
0,erer,0.0
1,Moscow,1.0
2,13op,0.0
3,Ilina,1.0
4,Tyumen,1.0
...,...,...
384,Serdyuk,1.0
385,Temchenko,1.0
386,Hitruk,1.0
387,Emelyanova,1.0


In [13]:
cdt=np.array(data.iloc[:,0])

In [7]:
!git clone https://github.com/Terkingil/Project_NLP.git

Cloning into 'Project_NLP'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 149 (delta 32), reused 8 (delta 1), pack-reused 66
Receiving objects: 100% (149/149), 67.87 MiB | 32.39 MiB/s, done.
Resolving deltas: 100% (53/53), done.


Создание токенов

In [14]:
#tokenize

num_char = 63 #
tokenizer=Tokenizer(num_words=num_char, lower=False,char_level=True,filters="'!#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n'")
tokenizer.fit_on_texts(cdt)
print(tokenizer.word_index)

inp_chars = 3
one_hot_enc=tokenizer.texts_to_sequences(cdt)


{'o': 1, 'a': 2, 'e': 3, 'k': 4, 'v': 5, 'r': 6, 'i': 7, 's': 8, 'n': 9, 'd': 10, 'l': 11, 'h': 12, 'u': 13, 't': 14, 'p': 15, 'j': 16, 'S': 17, 'y': 18, 'b': 19, 'm': 20, 'K': 21, 'g': 22, 'A': 23, 'c': 24, 'z': 25, 'P': 26, 'f': 27, 'M': 28, 'T': 29, 'B': 30, 'q': 31, 'N': 32, '[': 33, 'E': 34, 'w': 35, '3': 36, 'L': 37, 'V': 38, '2': 39, 'D': 40, 'G': 41, '1': 42, 'Z': 43, 'U': 44, 'R': 45, 'O': 46, ';': 47, 'F': 48, 'H': 49, 'I': 50, '-': 51, 'Y': 52, '4': 53, "'": 54, 'C': 55, ' ': 56, '/': 57, 'x': 58, 'J': 59, '.': 60, 'W': 61, '\\': 62, '9': 63}


Паддинг

In [ ]:
print(cdt[2])
print(one_hot_enc[2])



train_pded=pad_sequences(one_hot_enc, padding="post", truncating="post", maxlen=15)

print(train_pded[2])

13op
[42, 36, 1, 15]
[42 36  1 15  0  0  0  0  0  0  0  0  0  0  0]


Создание samples

In [ ]:

X=train_pded
y=np.array(data.iloc[:,1])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21) # Выборка небольшая поэтому 10% от выборки на тест

X_train.shape,y_train.shape


((350, 15), (350,))

# Модель

In [6]:
S_C_model=Sequential()
S_C_model.add(Embedding(63,15,input_length=15))
S_C_model.add(GRU(500, activation='tanh'))
S_C_model.add(Dense(1, activation='sigmoid'))#Полносвязный слой
S_C_model.summary()

S_C_model.compile(loss='binary_crossentropy', metrics=["AUC"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 15)            945       
                                                                 
 gru (GRU)                   (None, 500)               775500    
                                                                 
 dense (Dense)               (None, 1)                 501       
                                                                 
Total params: 776,946
Trainable params: 776,946
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=S_C_model.fit(X_train,y_train,batch_size=5,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
70/70 [==============================] - 7s 78ms/step - loss: 0.6559 - auc: 0.4937 - val_loss: 0.5703 - val_auc: 0.9954
Epoch 2/20
70/70 [==============================] - 6s 80ms/step - loss: 0.3470 - auc: 0.8898 - val_loss: 0.2004 - val_auc: 0.9815
Epoch 3/20
70/70 [==============================] - 5s 69ms/step - loss: 0.2131 - auc: 0.9425 - val_loss: 0.2073 - val_auc: 0.9815
Epoch 4/20
70/70 [==============================] - 5s 68ms/step - loss: 0.1896 - auc: 0.9527 - val_loss: 0.2811 - val_auc: 0.9722
Epoch 5/20
70/70 [==============================] - 5s 69ms/step - loss: 0.1445 - auc: 0.9646 - val_loss: 0.5297 - val_auc: 0.9738
Epoch 6/20
70/70 [==============================] - 5s 67ms/step - loss: 0.1380 - auc: 0.9749 - val_loss: 0.4610 - val_auc: 0.9738
Epoch 7/20
70/70 [==============================] - 5s 69ms/step - loss: 0.1312 - auc: 0.9778 - val_loss: 0.3549 - val_auc: 0.9738
Epoch 8/20
70/70 [==============================] - 5s 68ms/step - loss: 0.1048 - a

In [ ]:
S_C_model.save_weights('./S_C_model/S_C_weights')

# Проверка ручным вводом

In [10]:
S_C_model.load_weights('./Project_NLP/Models weights n data/S_C_model/S_C_weights')

In [16]:
a="St. Petersburg"
inp=[a]

inp_sname_tkzd=tokenizer.texts_to_sequences(inp)
#print(inp_enc)
inp_sname_tkzd_pded=pad_sequences(inp_sname_tkzd, padding="post", truncating="post", maxlen=15)

  #print(inp_enc_pded)

surname_hyppo = (S_C_model.predict(inp_sname_tkzd_pded))

surname_hyppo

1/1 [==============================] - 0s 32ms/step


array([[0.9984044]], dtype=float32)